In [1]:
import pandas as pd

In [2]:
raw_data_filename = 'rawdata/DEM_Challenge_Section1_DATASET.xlsx'

In [36]:
df = pd.read_excel(raw_data_filename)

In [4]:
df.head()

,id,first_name,last_name,email,gender,ip_address
0,1,Margaretta,Laughtisse,mlaughtisse0@mediafire.com,Genderfluid,34.148.232.131
1,2,Vally,Garment,vgarment1@wisc.edu,Bigender,15.158.123.36
2,3,Tessa,Curee,tcuree2@php.net,Bigender,132.209.143.225
3,4,Arman,Heineking,aheineking3@tuttocitta.it,Male,157.110.61.233
4,5,Roselia,Trustie,rtrustie4@ft.com,Non-binary,49.55.218.81


In [37]:
# df.describe(include='all')
pd.DataFrame({'count': df.count(), 'nunique': df.nunique()})

,count,nunique
id,1000,1000
first_name,1000,947
last_name,1000,989
email,1000,1000
gender,1000,8
ip_address,1000,1000


In [39]:
# Cleaning Data
# Trim whitespace and standardize case
df['first_name'] = df['first_name'].str.strip()
df['last_name'] = df['last_name'].str.strip()
df['email'] = df['email'].str.strip().str.lower()
df['gender'] = df['gender'].str.strip().str.lower()
df['ip_address'] = df['ip_address'].str.strip()

pd.DataFrame({'count': df.count(), 'nunique': df.nunique()})

,count,nunique
id,1000,1000
first_name,1000,947
last_name,1000,989
email,1000,1000
gender,1000,8
ip_address,1000,1000


In [31]:
# 3. Detect invalid emails
# Simple regex-based validation for email format
invalid_emails = df[~df['email'].str.contains(r'^[A-Za-z0-9._%+-]+@[A-Za-z0-9.-]+\.[A-Za-z]{2,}$', na=False)]
print("email:")
print(invalid_emails)

email:
Empty DataFrame
Columns: [id, first_name, last_name, email, gender, ip_address]
Index: []


In [33]:
# 4. Check for unique IP addresses
duplicate_ips = df[df.duplicated('ip_address', keep=False)]
print("Duplicate IP Addresses:")
print(duplicate_ips)

Duplicate IP Addresses:
Empty DataFrame
Columns: [id, first_name, last_name, email, gender, ip_address]
Index: []


In [43]:
print(df['gender'].value_counts())

gender
genderfluid    140
female         131
genderqueer    130
male           129
agender        120
bigender       119
polygender     118
non-binary     113
Name: count, dtype: int64


In [34]:
# 5. Standardize Gender Identity
df['gender'] = df['gender'].str.lower().map({
    'male': 'Male',
    'female': 'Female',
    'non-binary': 'Non-Binary',
}).fillna('Other')
print("Standardized Gender Identity:")
print(df['gender'].value_counts())

Standardized Gender Identity:
gender
Other     740
Female    131
Male      129
Name: count, dtype: int64


In [46]:
import re
def is_valid_ip(ip):
    pattern = r'^(\d{1,3}\.){3}\d{1,3}$'
    match = re.match(pattern, ip)
    if match:
        parts = ip.split('.')
        return all(0 <= int(part) <= 255 for part in parts)
    return False

valid_ips = df['ip_address'].apply(is_valid_ip)
invalid_ips = df[~valid_ips]
print("Invalid IP Addresses:")
print(invalid_ips)

Invalid IP Addresses:
Empty DataFrame
Columns: [id, first_name, last_name, email, gender, ip_address]
Index: []


In [50]:
# 8. Fuzzy Matching for similar full names (First name + Last name)
from fuzzywuzzy import fuzz, process
def find_similar_full_names(threshold=85):
    df['Full Name'] = df['first_name'] + ' ' + df['last_name']
    results = []
    for name in df['Full Name']:
        matches = process.extract(name, df['Full Name'], scorer=fuzz.token_sort_ratio)
        for match in matches:
            if match[1] >= threshold and match[0] != name:
                results.append((name, match[0], match[1]))
    return results

similar_full_names = find_similar_full_names()
print("Similar Full Names:")
for name1, name2, score in similar_full_names:
    print(f"{name1} <--> {name2} (Score: {score})")

Similar Full Names:


In [53]:
df.columns

Index(['id', 'first_name', 'last_name', 'email', 'gender', 'ip_address',
       'Full Name'],
      dtype='object')